This is where the magic happens:

# MODELLING

In this notebook I'll work on a model to what a tourist will spend when vacationing in Tanzania.
The evaluation metric for the model is **Mean Absolute Error**.


To do:
- remove total_male, night_mainland (multicollinearity)
- redo function: either also remove those rows from target variable or separate target variable later or not remove rows at all
- comments
- baseline model
- feature selection/feature engineering (subregions)
- model
- again if necessary: feature selection/feature engineering
- outlier handling (Isolation Forest?)
- hyperparameter tuning
- interpretation/visualization

In [12]:
# import some packages that I'll need

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, RobustScaler

# suppress warnings
import warnings
warnings.filterwarnings('ignore')

# set color scheme
cpal = ["#f94144","#f3722c","#f8961e","#f9844a","#f9c74f","#90be6d","#43aa8b","#4d908e","#577590","#277da1"]

# seaborn theme
sns.set()

# use natural numbers
pd.options.display.float_format = "{:.2f}".format

# set random seed
RSEED = 42

In [2]:
# load data
TZA = pd.read_csv('data/Train.csv')

In [3]:
# separate target variable

X = TZA.drop(['total_cost'], axis=1)
y = TZA['total_cost']

In [4]:
# train test split
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.3, random_state = RSEED)

## Preprocessing

I am going to preprocess the data now. I'll do it separately for train and test data, carefully avoiding data leakage.

### Missing values and minor adjustments

In [5]:
# function to handle missing data and to make some minor adjustments on the dataset

def adjustments(df):
    # fill NaN total_male/total_female with 0
    df['total_male'] = df['total_male'].fillna(0)
    df['total_female'] = df['total_female'].fillna(0)
    # add a column group_size based on total_male/total_female
    df['group_size'] = df['total_female'] + df['total_male']
    # fill NaN travel_with with "Alone" if group_size is zero
    df.loc[df.group_size == 1, 'travel_with'] = 'Alone'
    # fill remaining NaN travel_with with missing
    df['travel_with'] = df['travel_with'].fillna('missing')
    # fill NaN most_impressing with "No comments"
    df['most_impressing'] = df['most_impressing'].fillna('No comments')
    # add a column total_nights based on night_zanzibar/night_mainland
    df['total_nights'] = df['night_zanzibar'] + df['night_mainland']
    # delete rows if group_size is zero
    df = df[df.group_size > 0]
    # delete rows if total_nights is zero
    df = df[df.total_nights > 0]
    # drop id column
    df = df.drop(['ID'], axis =1)
    return df

In [6]:
# apply function on train data
X_train = adjustments(X_train)
# apply function on test data
X_test = adjustments(X_test)

### Build Pipelines

In [14]:
cat_features = list(X_train.columns[X_train.dtypes==object])
cat_pipeline = Pipeline([
    ('1hot', OneHotEncoder(handle_unknown= 'ignore', drop = 'first'))
])

In [15]:
num_features = list(X_train.columns[X_train.dtypes!=object])
num_pipeline = Pipeline([
    ('rob_scaler', RobustScaler())
])

In [16]:
preprocessor = ColumnTransformer([
    ('num', num_pipeline, num_features),
    ('cat', cat_pipeline, cat_features)
])